<a href="https://colab.research.google.com/github/greapis/Tensorflow_Python/blob/main/Tensorflow_AI_LS_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1. Library

 - Keras에서 일부 전처리된 형태의 IMDB Data 제공
 - Keras - sklearn이 서로 호환되지 않기 때문에 "KerasClassifier" 필요 : 이 부분은 Tensorflow가 업데이트되면서 변동되어서 확인 필요

In [10]:
# Importing Keras abd Sklearn libraries
    # Importing KerasClassifier Library : Keras vs Sklearn 호환성 이슈 해결

# IMDB Datasets 불러오기 from Keras
from tensorflow.keras.datasets import imdb

# 리뷰 길이 맞추기
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Making Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

# Keras model - (래핑) - sklearn이 이해할 수 있는 형태로 변환
from scikeras.wrappers import KerasClassifier
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier  : 최신 TensorFlow(특히 2.12 이상)에서는 tensorflow.keras.wrappers.scikit_learn 모듈이 더 이상 제공되지 않습니다.

# Importing GreidSearch
from sklearn.model_selection import GridSearchCV

# Caculating Accuracy
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'scikeras'

# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier :


*  최신 TensorFlow(특히 2.12 이상)에서는 tensorflow.keras.wrappers.scikit_learn 모듈이 더 이상 제공되지 않습니다.
*  과거에는 Keras가 scikit-learn과의 호환성을 위해 이 모듈을 제공했지만, 이제는 공식적으로 지원을 중단했으며, 이 기능은 별도의 패키지로 분리되었습니다



*   scikeras 사용 : 대안으로 scikeras라는 패키지를 사용할 수 있습니다.
다음처럼 임포트하면 됩니다:
*   from scikeras.wrappers import KerasClassifier 또는 from scikeras.wrappers import KerasRegressor




